# Import CH4 aerial observations data into unified database

## Resources
http://localhost:8888/notebooks/misc/arcgis-python-api/guide/04-feature-data-and-analysis/editing-features.ipynb

https://developers.arcgis.com/python/guide/accessing-and-creating-content/

http://localhost:8888/notebooks/misc/arcgis-python-api/samples/05_content_publishers/updating_features_in_a_feature_layer.ipynb

## Import libraries and setup

In [1]:
import os
import datetime
import json
import shutil
from copy import deepcopy

import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import arcgis
from arcgis.gis import GIS
from arcgis.features import GeoAccessor
from arcgis import geometry #use geometry module to project Long,Lat to X and Y
#import contextily as cx

In [3]:
# Define layer id for aerial observations unified database
aerial_obs_layer_id = '8df1e448e7424ad09d2a8a1521267692'

# Define layer id for MCBU facilities dataset
facilities_layer_id = '857b2093bff44132b39e65176c309284'

# Define path to csv for new aerial observations
csv = 'MCBU_aerial_obs_3_24_2022.csv'

## Function to load feature layer into GDF

In [4]:
def FeatureToGdf(feature_layer):
    """ Converts feature layer to geodataframe
    
    Parameters
    -----------
    feature_layer : ArcGIS API feature layer

    Returns
    -----------
    feature_gdf : geodataframe
    
    """
    import geopandas as gpd

    # .query() returns a FeatureSet
    fset = feature_layer.query()

    # get a GeoJSON string representation of the FeatureSet
    gjson_string = fset.to_geojson

    # read GeoJSON string into a dict
    gjson_dict = json.loads(gjson_string)

    # Read the dict into a gdf
    feature_gdf = gpd.GeoDataFrame.from_features(gjson_dict['features'])
    
    # Return GDF
    return feature_gdf   

## Connect to Chevron MapHub

In [5]:
gis = GIS("https://maphub-amer.tst.azure.chevron.com/portal", client_id='bb6vn14P957pN6bV')
print("Logged in as: " + gis.properties.user.username)

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://maphub-amer.tst.azure.chevron.com/portal/sharing/rest//oauth2/authorize?response_type=code&client_id=bb6vn14P957pN6bV&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=xP5NFOTIPZRDZCkkbeaqtVVrps6d6g
Enter code obtained on signing in using SAML: ········


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3-clone\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'maphub-amer.tst.azure.chevron.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings



Logged in as: ekqo


## Load CSV of new aerial observations

In [6]:
# Read csv into a df
aerial_obs_csv = pd.read_csv(csv)

# Read df into gdf and set crs
aerial_obs_gdf = gpd.GeoDataFrame(
    aerial_obs_csv, geometry=gpd.points_from_xy(aerial_obs_csv.obs_long, aerial_obs_csv.obs_lat))
aerial_obs_gdf = aerial_obs_gdf.set_crs(epsg=4267)
aerial_obs_gdf

,obs_id,obs_datetime,obs_lat,obs_long,obs_q,obs_sensor,geometry
0,9Y84LZ6,2/18/2022 16:49,32.2987,-104.0665,230,GHGSat-C2,POINT (-104.06650 32.29870)
1,9XS28ET_1,2/10/2022 16:43,31.2719,-103.3187,"2,766",GHGSat-C2,POINT (-103.31870 31.27190)
2,9XS28ET_2,2/10/2022 16:43,31.2720,-103.3115,"1,421",GHGSat-C2,POINT (-103.31150 31.27200)


## Check aerial observations against unified database

In [7]:
# Load the aerial observations unified database from MapHub
aerial_obs_item = gis.content.get(aerial_obs_layer_id)
aerial_obs_layers = aerial_obs_item.layers
aerial_obs_fset = aerial_obs_layers[0].query()
aerial_obs_fset.sdf

,objectid,fot_asset_name,child_obj_code,child_obj_desc,mcbu_gis_data_uploaded_date,child_lat,child_long,obs_id,obs_datetime,obs_lat,obs_long,obs_q,obs_sensor,SHAPE
0,2,TEST,TEST,TEST,2022-03-24 16:01:16.407000065,None,None,9Y84LZ6,2022-03-24 16:13:42.467000008,None,None,200.0,GHGSat,"{""x"": -11520140.3642, ""y"": 3681516.2709000036,..."


In [8]:
# Select those rows in aerial_obs_csv that are not in aerial_obs_fset using obs_id field
new_rows_gdf = aerial_obs_gdf[~aerial_obs_gdf['obs_id'].isin(aerial_obs_fset.sdf['obs_id'])]
new_rows_gdf

,obs_id,obs_datetime,obs_lat,obs_long,obs_q,obs_sensor,geometry
1,9XS28ET_1,2/10/2022 16:43,31.2719,-103.3187,"2,766",GHGSat-C2,POINT (-103.31870 31.27190)
2,9XS28ET_2,2/10/2022 16:43,31.2720,-103.3115,"1,421",GHGSat-C2,POINT (-103.31150 31.27200)


## Load new observations into unified database

### Perform spatial join of new observations against MCBU facilities data

In [9]:
# Load the MCBU facilities data into a GDF
mcbu_facilities = gis.content.get(facilities_layer_id)

# Create blank list to store results
mcbu_facilities_list = []

# Iterate over the layers in the collection
for layer in mcbu_facilities.layers:

    # Check layer type to only process points
    if layer.properties.geometryType == 'esriGeometryPoint':
        
        # Convert the feature layer to a GDF
        facilities_gdf = FeatureToGdf(layer)

        # Append the gdf to the list
        mcbu_facilities_list.append(facilities_gdf)
        
    else:
        continue
    
# Combine all the df's into a single df for all the years
mcbu_facilities_gdf = pd.concat(mcbu_facilities_list, axis=0)

# Format the date fields as datetime
mcbu_facilities_gdf['MCBU_GIS_DATA_RECEIVED_DATE'] = pd.to_datetime(
    mcbu_facilities_gdf['MCBU_GIS_DATA_RECEIVED_DATE'], unit='ms')

mcbu_facilities_gdf['MCBU_GIS_DATA_UPLOADED_DATE'] = pd.to_datetime(
    mcbu_facilities_gdf['MCBU_GIS_DATA_UPLOADED_DATE'], unit='ms')

# Set the CRS
mcbu_facilities_gdf = mcbu_facilities_gdf.set_crs(epsg=4267)
#mcbu_facilities_gdf

In [10]:
# Use spatial join to find nearest facility to each aerial observation
new_rows_joined_gdf = gpd.sjoin_nearest(new_rows_gdf, mcbu_facilities_gdf, how='inner', max_distance=0.01)
new_rows_joined_gdf.style

AttributeError: module 'geopandas' has no attribute 'sjoin_nearest'

### Create features from new rows

In [ ]:
# Get an example feature
features_for_update = aerial_obs_fset.features[0]

# Get a template feature object
template_feature = deepcopy(features_for_update)

# List of features to be added
features_to_be_added = []

# loop through each row and add to the list of features to be added
for row in new_rows_joined_gdf.iterrows():
    new_feature = deepcopy(template_feature)

    # print
    print("Creating entry for obs_id: " + row[1]['obs_id'])

    # get geometries in the destination coordinate system
    input_geometry = {'y': float(row[1]['CHILD_LAT']),
                      'x': float(row[1]['CHILD_LONG'])}
    output_geometry = geometry.project(geometries=[input_geometry],
                                       in_sr=new_rows_joined_gdf.crs,
                                       out_sr=aerial_obs_fset.spatial_reference['latestWkid'],
                                       gis=gis)

In [ ]:
# Loop over the rows to populate featureset

In [ ]:
# Get data
mcbu_facilities = gis.content.get(facilities_layer_id)

# Create blank list to store results
mcbu_facilities_list = []

# Iterate over the layers in the collection
for layer in mcbu_facilities.layers:

    # Check layer type to only process points
    if layer.properties.geometryType == 'esriGeometryPoint':
        
        # Convert the feature layer to a GDF
        facilities_gdf = FeatureToGdf(layer)

        # Append the gdf to the list
        mcbu_facilities_list.append(facilities_gdf)
        
    else:
        continue
    
# Combine all the df's into a single df for all the years
mcbu_facilities_df = pd.concat(mcbu_facilities_list, axis=0)

# Format the date fields as datetime
mcbu_facilities_df['MCBU_GIS_DATA_RECEIVED_DATE'] = pd.to_datetime(
    mcbu_facilities_df['MCBU_GIS_DATA_RECEIVED_DATE'], unit='ms')

mcbu_facilities_df['MCBU_GIS_DATA_UPLOADED_DATE'] = pd.to_datetime(
    mcbu_facilities_df['MCBU_GIS_DATA_UPLOADED_DATE'], unit='ms')

# Set the CRS
mcbu_facilities_df = mcbu_facilities_df.set_crs(epsg=4267)
mcbu_facilities_df

## Load the MCBU_CH4_aerial_obs feature layer